## torch.tensor vs. torch.Tensor

In [236]:
import torch
i = torch.tensor(1)  # gives you a tensor with fixed value of 1
j = torch.Tensor(1)  # gives you a tensor of size 1 which is randomly initialized
i, j

(tensor(1), tensor([0.]))

In [185]:
import sys
a = 45   # int64
b = 45.2 # float64
print(type(a))
print(type(b))
print(sys.getsizeof(a)) # prints 28 bytes
print(sys.getsizeof(b)) # prints 24 bytes

# https://stackoverflow.com/questions/31469553/python-memory-consumption-in-64-bit-system-for-int-and-float
# 64 bit system
# int:   overhead = 20 bytes, value = 8 bytes
# float: overhead = 16 bytes, value = 8 bytes

<class 'int'>
<class 'float'>
28
24


In [242]:
import numpy as np
n_int = np.array([1, 2, 3])         # int64
n_float = np.array([1.0, 2.0, 3.0]) # float64
print(n_int.dtype)
print(n_float.dtype)

print(n_int.size)
print(n_float.size)

print(n_int.itemsize)
print(n_float.itemsize)

print(n_int.size * n_int.itemsize)
print(n_float.size * n_float.itemsize)

print(n_int.nbytes)
print(n_float.nbytes)

int64
float64
3
3
8
8
24
24
24
24


In [245]:
import torch
t_int = torch.tensor([1, 2, 3])                                 # int64
t_float_1 = torch.tensor([1.0, 2.0, 3.0])                       # flaot32 !!!!!!
t_float_2 = torch.tensor([1.0, 2.0, 3.0], dtype=torch.float64)  # flaot64 !!!!!!
print(t_int.dtype)
print(t_float_1.dtype)
print(t_float_2.dtype)

print(t_int.element_size())
print(t_float_1.element_size())
print(t_float_2.element_size())

print(t_int.nelement())
print(t_float_1.nelement())
print(t_float_2.nelement())

print(t_int.element_size() * t_int.nelement())
print(t_float_1.element_size() * t_float_1.nelement())
print(t_float_2.element_size() * t_float_2.nelement())

torch.int64
torch.float32
torch.float64
8
4
8
3
3
3
24
12
24


## view vs. transpose vs. reshape
- https://inmoonlight.github.io/2021/03/03/PyTorch-view-transpose-reshape/

In [211]:
t = torch.tensor([[[0, 1],
                   [2, 3],
                   [4, 5]],
                  [[6, 7],
                   [8, 9],
                   [10, 11]],
                  [[12, 13],
                   [14, 15],
                   [16, 17]],
                  [[18, 19],
                   [20, 21],
                   [22, 23]]])  # (4, 3, 2)
t.shape

torch.Size([4, 3, 2])

In [212]:
tv = t.view(4, 2, 3)
tv

tensor([[[ 0,  1,  2],
         [ 3,  4,  5]],

        [[ 6,  7,  8],
         [ 9, 10, 11]],

        [[12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23]]])

In [213]:
tv.shape

torch.Size([4, 2, 3])

In [207]:
tv.is_contiguous()

True

In [214]:
t = torch.tensor([[[0, 1],
                   [2, 3],
                   [4, 5]],
                  [[6, 7],
                   [8, 9],
                   [10, 11]],
                  [[12, 13],
                   [14, 15],
                   [16, 17]],
                  [[18, 19],
                   [20, 21],
                   [22, 23]]])  # (4, 3, 2)
t.shape

torch.Size([4, 3, 2])

#### transpose
- tensor 에 저장된 데이터의 물리적 위치 순서와 상관없이 수학적 의미의 transpose를 수행한다.
- 따라서, 일반적으로 transpose가 수행된 tensor는 항상 contiguous 하지 않다.

In [221]:
tr = t.transpose(2, 1)
tr.shape

torch.Size([4, 2, 3])

In [222]:
tr

tensor([[[ 0,  2,  4],
         [ 1,  3,  5]],

        [[ 6,  8, 10],
         [ 7,  9, 11]],

        [[12, 14, 16],
         [13, 15, 17]],

        [[18, 20, 22],
         [19, 21, 23]]])

In [219]:
tr.is_contiguous()

False

#### view 는 contiguous 하지 않은 tensor에 대해서는 적용할 수 없다.

In [226]:
tr.view(4, 3, 2)

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [227]:
tr2 = tr.contiguous()
tr2

tensor([[[ 0,  2,  4],
         [ 1,  3,  5]],

        [[ 6,  8, 10],
         [ 7,  9, 11]],

        [[12, 14, 16],
         [13, 15, 17]],

        [[18, 20, 22],
         [19, 21, 23]]])

In [229]:
tr3 = tr2.view(4, 3, 2)
tr3

tensor([[[ 0,  2],
         [ 4,  1],
         [ 3,  5]],

        [[ 6,  8],
         [10,  7],
         [ 9, 11]],

        [[12, 14],
         [16, 13],
         [15, 17]],

        [[18, 20],
         [22, 19],
         [21, 23]]])

#### tensor.reshape() == tensor.contiguous().view()
- reshape 은 강제로 tensor를 contiguous 하게 만들면서 shape을 변경

In [230]:
tr4 = tr.view(4, 3, 2)

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [233]:
tr4 = tr.reshape(4, 3, 2)
tr4

tensor([[[ 0,  2],
         [ 4,  1],
         [ 3,  5]],

        [[ 6,  8],
         [10,  7],
         [ 9, 11]],

        [[12, 14],
         [16, 13],
         [15, 17]],

        [[18, 20],
         [22, 19],
         [21, 23]]])

## Memory: List vs. Tensor

![...](https://raw.githubusercontent.com/HiddenBeginner/hiddenbeginner.github.io/master/static/img/_posts/2020-1-21-pytorch_tensor/figure2.png)

In [1]:
a0 = 1.0
a1 = 2.0
a2 = 3.0

print(id(a0) , id(a1), id(a2))

4393155056 4396643056 4396634992


In [2]:
a = [a0, a1, a2]

print(id(a[0]) , id(a[1]), id(a[2]))
print(id(a))

4393155056 4396643056 4396634992
4397599744


In [3]:
import torch
b = torch.tensor([a0, a1, a2])

print(id(b[0]), id(b[1]), id(b[2]))
print(id(b))

4767668704 4767668704 4767668704
4498415088


## Efficient Memory Management

In [4]:
data = [[1.0, 2.0],
        [3.0, 4.0],
        [5.0, 6.0]]
a = torch.tensor(data)
print(a)

tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])


In [5]:
b = a[1,:]
print(b)

tensor([3., 4.])


In [6]:
b[0] = -999
print(b)
print(a)

tensor([-999.,    4.])
tensor([[   1.,    2.],
        [-999.,    4.],
        [   5.,    6.]])


## Basic Tensor

In [7]:
a = [1.0, 2.0, 1.0]

In [8]:
a[0]

1.0

In [9]:
a[2] = 3.0
a

[1.0, 2.0, 3.0]

In [10]:
import torch # <1>
a = torch.ones(3) # <2>
a

tensor([1., 1., 1.])

In [11]:
a[1]

tensor(1.)

In [12]:
float(a[1])

1.0

In [13]:
a[2] = 2.0
a

tensor([1., 1., 2.])

In [14]:
points = torch.zeros(6) # <1>
points[0] = 4.0 # <2>
points[1] = 1.0
points[2] = 5.0
points[3] = 3.0
points[4] = 2.0
points[5] = 1.0

In [15]:
points = torch.tensor([4.0, 1.0, 5.0, 3.0, 2.0, 1.0])
points

tensor([4., 1., 5., 3., 2., 1.])

In [16]:
float(points[0]), float(points[1])

(4.0, 1.0)

In [17]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [18]:
points.shape

torch.Size([3, 2])

In [19]:
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [20]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [21]:
points[0, 1]

tensor(1.)

In [22]:
points[0]

tensor([4., 1.])

## Storage, Offset, Stride
- https://hiddenbeginner.github.io/deeplearning/2020/01/21/pytorch_tensor.html
- 1차원 텐서이든, 2차원 텐서이든 혹은 d차원 텐서이든 모든 텐서는 메모리에 1차원 배열의 형태로 저장됨
- storage: 텐서를 효율적으로 다룰 수 있도록 만들어주는 이러한 1차원 배열 (인덱싱한 텐서의 값에 접근할 때 offset과 stride 사용)
![...](https://raw.githubusercontent.com/HiddenBeginner/hiddenbeginner.github.io/master/static/img/_posts/2020-1-21-pytorch_tensor/figure3.png)
  - 위 두 텐서에 대하여 storage의 offset은 모두 0
  - stride는 텐서 안의 어떤 한 값에서 다음 원소를 얻기 위하여 storage에서 뛰어 넘어야할 인덱스의 개수
    - storage의 stride는 텐서의 차원만큼 정의됨

In [23]:
a = torch.tensor([[3,1,8],
                  [0,9,2]])
print("============storage==========")
print(a.storage())
print("=============================")
print("Offset: ", a.storage_offset())
print("Strides: ", a.stride())

============storage==========
 3
 1
 8
 0
 9
 2
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]
Offset:  0
Strides:  (3, 1)


/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/800149035.py:4: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  print(a.storage())
/Users/yhhan/miniforge3/envs/link_pytorch/lib/python3.10/site-packages/torch/storage.py:636: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  f'device={self.device}) of size {len(self)}]')
/Users/yhhan/miniforge3/envs/link_pytorch/lib/python3.10/site-packages/torch/storage.py:637: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage 

In [24]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/3297796694.py:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points.storage()
/Users/yhhan/miniforge3/envs/link_pytorch/lib/python3.10/site-packages/IPython/lib/pretty.py:778: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  output = repr(obj)
/Users/yhhan/miniforge3/envs/link_pytorch/lib/python3.10/site-packages/torch/storage.py:645: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

### tenset[i, j] == storage[storage_offset + i * stirde[0] + j * stirde[1]]
### tenset[i, j, k] == storage[storage_offset + i * stirde[0] + j * stirde[1] + k * stirde[2]]

In [25]:
x1 = torch.arange(12).view((4, 3))
x1

tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]])

In [26]:
x1.stride()

(3, 1)

In [27]:
x2 = x1.view(3, 4)
x2

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [28]:
storage = x2.storage()
offset = x2.storage_offset()
stride = x2.stride()
offset, stride

/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/2939581920.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = x2.storage()


(0, (4, 1))

In [29]:
item1 = x2[2, 1].item()
item1

9

In [30]:
item2 = storage[offset + 2 * x2.stride()[0] + 1 * x2.stride()[1]]
item2

/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/250354989.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  item2 = storage[offset + 2 * x2.stride()[0] + 1 * x2.stride()[1]]


9

In [31]:
x = torch.arange(1200).view((10, 30, 4))
storage = x.storage()
offset = x.storage_offset()
stride = x.stride()
offset, stride

/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/894859101.py:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = x.storage()


(0, (120, 4, 1))

In [32]:
item1 = x[4, 23, 2].item()
item2 = storage[offset + 4 * stride[0] + 23 * stride[1] + 2 * stride[2]]
item1 == item2

/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/3116659906.py:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  item2 = storage[offset + 4 * stride[0] + 23 * stride[1] + 2 * stride[2]]


True

![...](https://raw.githubusercontent.com/HiddenBeginner/hiddenbeginner.github.io/master/static/img/_posts/2020-1-21-pytorch_tensor/figure4.png)

In [198]:
a = torch.tensor([[3,1,1,2],
                  [8,0,3,4],
                  [9,2,5,6]])
print("============storage==========")
print(a.storage())
print("=============================")
print("Offset: ", a.storage_offset())
print("Strides: ", a.stride())

============storage==========
 3
 1
 1
 2
 8
 0
 3
 4
 9
 2
 5
 6
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 12]
Offset:  0
Strides:  (4, 1)


/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/822766986.py:5: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  print(a.storage())
/Users/yhhan/miniforge3/envs/link_pytorch/lib/python3.10/site-packages/torch/storage.py:636: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  f'device={self.device}) of size {len(self)}]')
/Users/yhhan/miniforge3/envs/link_pytorch/lib/python3.10/site-packages/torch/storage.py:637: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage 

In [199]:
b = a[1:3, 1:3]
print("============storage==========")
print(b.storage())
print("=============================")
print("Offset: ", b.storage_offset())
print("Strides: ", b.stride())
storage = b.storage()
offset = b.storage_offset()
stride = b.stride()

============storage==========
 3
 1
 1
 2
 8
 0
 3
 4
 9
 2
 5
 6
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 12]
Offset:  5
Strides:  (4, 1)


/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/692778318.py:3: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  print(b.storage())
/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/692778318.py:7: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = b.storage()


In [200]:
id(a.storage()) == id(b.storage())

/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/1294772978.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  id(a.storage()) == id(b.storage())


False

In [201]:
id(a.untyped_storage()) == id(b.untyped_storage())

True

In [35]:
b

tensor([[0, 3],
        [2, 5]])

In [36]:
item1 = b[1, 1].item()
item1

5

In [37]:
item2 = storage[offset + 1 * stride[0] + 1 * stride[1]]
item1 == item2

/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/2310448803.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  item2 = storage[offset + 1 * stride[0] + 1 * stride[1]]


True

In [38]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_storage = points.storage()
points_storage[0]

/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/1412174013.py:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points_storage = points.storage()
/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/1412174013.py:3: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points_storage[0]


4.0

In [39]:
points.storage()[1]

/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/682517886.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points.storage()[1]


1.0

In [40]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_storage = points.storage()
points_storage[0] = 2.0
points

/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/3332447618.py:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points_storage = points.storage()
/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/3332447618.py:3: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points_storage[0] = 2.0


tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

In [41]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.stride()

(2, 1)

In [42]:
second_point = points[1]
second_point

tensor([5., 3.])

In [43]:
second_point.storage_offset()

2

In [44]:
second_point.size()

torch.Size([2])

In [45]:
second_point.shape

torch.Size([2])

In [46]:
second_point.stride()

(1,)

In [47]:
item1 = points[1, 1].item()
item1

3.0

In [48]:
item2 = second_point[1].item()
item2

3.0

In [49]:
item3 = points_storage[second_point.storage_offset() + 1 * second_point.stride()[0]]
item3

/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/91671260.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  item3 = points_storage[second_point.storage_offset() + 1 * second_point.stride()[0]]


3.0

In [50]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point[0] = 10.0
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

In [51]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [52]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [53]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [54]:
id(points.storage()) == id(points_t.storage())

/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/2277798614.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  id(points.storage()) == id(points_t.storage())


False

In [55]:
id(points.untyped_storage()) == id(points_t.untyped_storage())

True

In [56]:
points.stride()

(2, 1)

In [57]:
points_t.stride()

(1, 2)

In [58]:
some_t = torch.ones(3, 4, 5)
some_t.shape

torch.Size([3, 4, 5])

In [59]:
transpose_t = some_t.transpose(0, 2)
transpose_t.shape

torch.Size([5, 4, 3])

In [60]:
some_t.stride()

(20, 5, 1)

In [61]:
transpose_t.stride()

(1, 5, 20)

In [62]:
points.is_contiguous()

True

In [63]:
points_t.is_contiguous()

False

In [64]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [65]:
points_t.storage()

/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/2245626776.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points_t.storage()


 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [66]:
points_t.stride()

(1, 2)

In [67]:
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [68]:
points_t_cont.stride()

(3, 1)

In [69]:
points_t_cont.storage()

/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_43730/1075489549.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points_t_cont.storage()


 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [70]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)

In [71]:
short_points.dtype

torch.int16

In [72]:
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()

In [73]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype=torch.short)

In [74]:
points_64 = torch.rand(5, dtype=torch.double)  # <1>
points_short = points_64.to(torch.short)
points_64 * points_short  # works from PyTorch 1.3 onwards

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

In [75]:
some_list = list(range(6))
some_list[:]     # <1>

[0, 1, 2, 3, 4, 5]

In [76]:
some_list[1:4]   # <2>

[1, 2, 3]

In [77]:
some_list[1:]    # <3>

[1, 2, 3, 4, 5]

In [78]:
some_list[:4]    # <4>

[0, 1, 2, 3]

In [79]:
some_list[:-1]   # <5>

[0, 1, 2, 3, 4]

In [80]:
some_list[1:4:2] # <6>

[1, 3]

In [131]:
# reset points back to original value
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points[1:]       # <1>

tensor([[5., 3.],
        [2., 1.]])

In [132]:
points[1:, :]    # <2>

tensor([[5., 3.],
        [2., 1.]])

In [133]:
points[1:, 0]    # <3>

tensor([5., 2.])

In [134]:
points[None]     # <4>

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

## Working with NumPy

In [298]:
points = torch.ones(3, 4)
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [299]:
id(points) == id(points_np)

False

- points & points_np are objects of different Python types, so, as far as Python is concerned, they have different addresses
- However, they share an underlying C array in memory & modifying one in-place also modies the other

In [300]:
points.zero_()
points

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [301]:
print(points_np)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [302]:
points2 = torch.from_numpy(points_np)

In [303]:
points2.add_(10)
points2

tensor([[10., 10., 10., 10.],
        [10., 10., 10., 10.],
        [10., 10., 10., 10.]])

In [304]:
points_np

array([[10., 10., 10., 10.],
       [10., 10., 10., 10.],
       [10., 10., 10., 10.]], dtype=float32)

In [305]:
points

tensor([[10., 10., 10., 10.],
        [10., 10., 10., 10.],
        [10., 10., 10., 10.]])

## torch save & load

In [137]:
torch.save(points, '../data/p1ch3/ourpoints.t')

In [138]:
with open('../data/p1ch3/ourpoints.t','wb') as f:
   torch.save(points, f)

In [139]:
points = torch.load('../data/p1ch3/ourpoints.t')
points

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [157]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [158]:
with open('../data/p1ch3/ourpoints.t','rb') as f:
   points = torch.load(f)

In [159]:
#  pip install h5py
import h5py

f = h5py.File('../data/p1ch3/ourpoints.hdf5', 'w')
dset = f.create_dataset('coords', data=points.numpy())
f.close()
dset

OSError: Unable to create file (unable to truncate a file which is already open)

In [160]:
f = h5py.File('../data/p1ch3/ourpoints.hdf5', 'r')
dset = f['coords']
dset

<HDF5 dataset "coords": shape (3, 2), type "<f4">

In [161]:
last_points = torch.from_numpy(dset[1:])
f.close()

In [162]:
last_points

tensor([[5., 3.],
        [2., 1.]])

In [163]:
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device='cuda')

AssertionError: Torch not compiled with CUDA enabled

In [164]:
points_gpu = points.to(device='cuda')

AssertionError: Torch not compiled with CUDA enabled

In [165]:
points_gpu = points.to(device='cuda:0')

AssertionError: Torch not compiled with CUDA enabled

In [166]:
points = 2 * points  # <1>
points_gpu = 2 * points.to(device='cuda')  # <2>

AssertionError: Torch not compiled with CUDA enabled

In [167]:
points_gpu = points_gpu + 4

NameError: name 'points_gpu' is not defined

In [168]:
points_cpu = points_gpu.to(device='cpu')

NameError: name 'points_gpu' is not defined

In [169]:
points_gpu = points.cuda()  # <1>
points_gpu = points.cuda(0)
points_cpu = points_gpu.cpu()

AssertionError: Torch not compiled with CUDA enabled

In [170]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [171]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [172]:
a = torch.ones(3, 2)

In [173]:
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

## torch.dot vs. torch.matmul vs. torch.mm vs. torch.bmm
- https://velog.io/@regista/torch.dot-torch.matmul-torch.mm-torch.bmm

In [312]:
batch1 = torch.randn(10, 3, 20, 10)
batch2 = torch.randn(10, 3, 10, 30)
res = torch.matmul(batch1, batch2)
res.size() # torch.Size([10, 3, 20, 30])

torch.Size([10, 3, 20, 30])

In [313]:
batch1 = torch.randn(10, 3, 20, 10)
batch2 = torch.randn(10, 3, 10, 30)
res = batch1 @ batch2
res.size() # torch.Size([10, 3, 20, 30])

torch.Size([10, 3, 20, 30])

In [316]:
batch1 = torch.randn(10, 20, 10)
batch2 = torch.randn(10, 10, 30)
res = torch.bmm(batch1, batch2)
res.size() # torch.Size([10, 3, 20, 30])

torch.Size([10, 20, 30])

- https://pytorch.org/docs/stable/generated/torch.bmm.html
- mat1 and mat2 must be 3-D tensors